# ColBERT training

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

In [6]:
# Reimporting the whole dataset with results of T5 predictions to choose positive and negative examples
import ast
import pandas as pd
df_T5_100 = pd.read_csv('df_T5_top100.csv')
df_T5_100['T5Top100Passages'] = df_T5_100['T5Top100Passages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['T5posPassages'] = df_T5_100['T5posPassages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['T5negPassages'] = df_T5_100['T5negPassages'].apply(lambda x: ast.literal_eval(x))
df_T5_100['passages'] = df_T5_100['passages'].apply(lambda x: ast.literal_eval(x))
#df_T5_100['answerPage'] = df_T5_100['answerPage'].apply(lambda x: ast.literal_eval(x))

In [22]:
# Recomputing Queries with latest version
def extract_optionHelp2(optionHelp):
    optionHelp = str(optionHelp)
    result = re.search(r'(?<=The document\(s\) should demonstrate the following: )\s*.*',optionHelp)
    if result:
        return result.group(0).lower()
    else:
        return ''
    
def select_query2(option_help, option_text, question_text, question_type):
    option_help = extract_optionHelp2(str(option_help))
    if len(option_help) != 0:
        query = option_help[:500]
        query_type = 'option_help'
        
    elif question_type == 2:
        query = question_text
        query_type = 'question_text'
    elif len(str(option_text).split()) > 4 and option_text != 'topics other than those mentioned above (please specify)':
        query = str(option_text).lower()
        query_type = 'option_text'
    else:
        query = str(question_text).lower() + ' ' + str(option_text).lower()
        query_type = 'concat_quest_opt'
        
    return (query, query_type)

df_T5_100['query'] = df_T5_100.apply(lambda x: select_query2(x['optionHelp'], x['textOption'], x['textQuestion'], x['idQuestionType'])[0], axis=1)
df_T5_100['query_type'] = df_T5_100.apply(lambda x: select_query2(x['optionHelp'], x['textOption'], x['textQuestion'], x['idQuestionType'])[1], axis=1)

In [23]:
# Recomputing answerPage with latest version
def removeFakePgNbrs(pageNbr):
    pageNbr = pageNbr.lower().replace("l0", "").replace("l-1", "").replace("l1", "")
    return pageNbr
    

def removeYears(pageNbr):
    l = re.findall(r'\d{4}', pageNbr)
    if len(l)!=0:
        for s in l:
            pageNbr = pageNbr.replace(s, '')
    return pageNbr


def getPagesWhenRange(pageNbr):
    pgs = re.search(r'\d+\s*-\s*\d+', pageNbr).group().split("-")
    start = int(pgs[0])-1
    end = int(pgs[1])
    range_nbr = list(range(start, end))
    others = re.findall(r'\d+', pageNbr)
    others = [int(i)-1 for i in others]
    all_pgs = range_nbr + others
    return list(set(all_pgs))
    

def cleanPageNbr2(pageNbr):
    """
    :param pageNbr: non-standard text introduced by analyst
    :return: list of string : cleaned page numbers
    """
    pageNbr = str(pageNbr)
    pageNbr = removeFakePgNbrs(pageNbr)
    pageNbr = removeYears(pageNbr)
    if re.findall(r'\d+\s*-\s*\d+', pageNbr):
        pages = getPagesWhenRange(pageNbr)
        return pages 
    pages = re.findall(r'\d+', pageNbr)
    pages = [int(i)-1 for i in pages if i!= 0]
    return list(set(pages))

df_T5_100['answerPage'] = df_T5_100.apply(lambda x: cleanPageNbr2(x['analystSource']), axis=1)

In [7]:
#df_colbert = df_joined_veryshort.copy()
df_colbert = df_T5_100.copy()
df_colbert['query_id'] = df_colbert.index.values.tolist()
df_colbert

,index,idOption,idQuestionUnique,textOption,optionHelp,textQuestion,idQuestionType,optionCode,idQuestion,questionCode,...,query_type,query_id,queryWordCount,passages,pygaggleQuery,pygagglePassages,rerankedPassages,T5Top100Passages,T5posPassages,T5negPassages
0,3,2342,300,Provision of fuel consumption reduction inform...,Examples of documents to attach: product/servi...,Does your company provide services to clients ...,1,ENV420_10,300,ENV420,...,option_help,0,24,"[[0, 0, —toward the toyota environmental chall...",<pygaggle.rerank.base.Query object at 0x7efeea...,[<pygaggle.rerank.base.Text object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe94...,"[[1, 162, -0.09654336422681808, data of the re...","[[2, 152, -2.076173782348633, stage. in this s...","[[1, 162, -0.09654336422681808, data of the re..."
1,4,9666,1202,Reduction of water consumption through employe...,Examples of documents to attach: training mate...,What actions are in place regarding water mana...,1,ENV321_10,28,ENV321,...,option_help,1,16,"[[0, 406, —toward the toyota environmental cha...",<pygaggle.rerank.base.Query object at 0x7efeea...,[<pygaggle.rerank.base.Text object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe94...,"[[1, 768, -0.02047719992697239, ensure that al...","[[37, 755, -9.55618953704834, to protect human...","[[1, 768, -0.02047719992697239, ensure that al..."
2,5,9667,1202,Reduction of water consumption through innovat...,Examples of documents to attach: standard oper...,What actions are in place regarding water mana...,1,ENV321_20,28,ENV321,...,option_help,2,8,"[[0, 812, —toward the toyota environmental cha...",<pygaggle.rerank.base.Query object at 0x7efeea...,[<pygaggle.rerank.base.Text object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe94...,"[[1, 862, -0.6157100796699524, at newly establ...","[[7, 1027, -8.945672988891602, makes it impera...","[[1, 862, -0.6157100796699524, at newly establ..."
3,6,10514,1300,Integration of ecodesign features in product d...,Examples of documents to attach: design specif...,What actions are in place to reduce the enviro...,1,ENV386_01,294,ENV386,...,option_help,3,29,"[[0, 1218, —toward the toyota environmental ch...",<pygaggle.rerank.base.Query object at 0x7efeea...,[<pygaggle.rerank.base.Text object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe94...,"[[1, 1370, -0.20025764405727386, stage. in thi...","[[11, 1359, -5.017537593841553, to curb global...","[[1, 1370, -0.20025764405727386, stage. in thi..."
4,7,10515,1300,Environmental impact analyses performed (e.g. ...,Examples of documents to attach: carbon footpr...,What actions are in place to reduce the enviro...,1,ENV386_02,294,ENV386,...,option_help,4,51,"[[0, 1624, —toward the toyota environmental ch...",<pygaggle.rerank.base.Query object at 0x7efeea...,[<pygaggle.rerank.base.Text object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe94...,"[[1, 1776, -0.015775874257087708, stage. in th...","[[4, 1692, -0.03989306464791298, will contribu...","[[1, 1776, -0.015775874257087708, stage. in th..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8345,32066,18512,2348,Health care coverage of employees in place,Examples of documents to attach: detailed heal...,What actions are in place regarding working co...,1,LAB3201_6,1737,LAB3201,...,option_help,8345,24,"[[0, 2445552, 1 employee handbook 2019], [1, 2...",<pygaggle.rerank.base.Query object at 0x7efed4...,[<pygaggle.rerank.base.Text object at 0x7efe98...,[<pygaggle.rerank.base.Text object at 0x7efd3c...,"[[1, 2445560, -0.6540103554725647, group healt...","[[2, 2445562, -1.0587340593338013, the first 3...","[[1, 2445560, -0.6540103554725647, group healt..."
8346,32067,18509,2348,Additional leave beyond standard vacation days,Examples of documents to attach: employee hand...,What actions are in place regarding working co...,1,LAB3201_3,1737,LAB3201,...,option_help,8346,22,"[[0, 2445724, handbook employee], 

### Building ColBERT training set

In [25]:
# Recommended format: TSV file with query text \t positive passage text \t negative passage text
import csv
import random

# Creating a dataset with triples: query, positive passage, negative passage

rows_list = []
count = 0
for row in range(len(df_colbert)):
    for i in range(min(3,len(df_colbert.T5posPassages.iloc[row]))):
        neg_samples_idx = random.sample(range(len(df_colbert.T5negPassages.iloc[row])), min(10, len(df_colbert.T5negPassages.iloc[row])))
        for j in neg_samples_idx:
            dict_example = {}
            dict_example.update({'query': df_colbert['query'].iloc[row]}) 
            dict_example.update({'query_id': df_colbert['query_id'].iloc[row]}) 
            dict_example.update({'positive': df_colbert['T5posPassages'].iloc[row][i][3]})
            dict_example.update({'negative': df_colbert['T5negPassages'].iloc[row][j][3]})
            dict_example.update({'example_id': count}) 
            count += 1
            rows_list.append(dict_example)

df_triple_full = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'positive', 'negative', 'example_id'])
df_triple_full['query_pos_neg'] = df_triple_full.apply(lambda x: [x['query'], x['positive'], x['negative']], axis=1)

In [26]:
df_triple_full

,query,query_id,positive,negative,example_id,query_pos_neg
0,detailed information available on fuel consump...,0,"stage. in this system, we conduct lca* which a...",cumulative sales (global) third party assuranc...,0,[detailed information available on fuel consum...
1,detailed information available on fuel consump...,0,"stage. in this system, we conduct lca* which a...",in harmony with nature environmental managemen...,1,[detailed information available on fuel consum...
2,detailed information available on fuel consump...,0,"stage. in this system, we conduct lca* which a...",space on trucks going from the port to the pla...,2,[detailed information available on fuel consum...
3,detailed information available on fuel consump...,0,"stage. in this system, we conduct lca* which a...",to convey information about toyota’s initiativ...,3,[detailed information available on fuel consum...
4,detailed information available on fuel consump...,0,"stage. in this system, we conduct lca* which a...",editorial policy contents overview highlights ...,4,[detailed information available on fuel consum...
...,...,...,...,...,...,...
194813,monetary remuneration for employees beyond bas...,8349,again committed to link reward more closely wi...,year there will be an equal weighting between ...,194813,[monetary remuneration for employees beyond ba...
194814,monetary remuneration for employees beyond bas...,8349,again committed to link reward more closely wi...,be more agile and individually. to deliver our...,194814,[monetary remuneration for employees beyond ba...
194815,monetary remuneration for employees beyond bas...,8349,2019 stvrs gkn powder metallurgy may 2019 2019...,year there will be an equal weighting between ...,194815,[monetary remuneration for employees beyond ba...
194816,monetary remuneration for employees beyond bas...,8349,2019 stvrs gkn powder metallurgy may 2019 2019...,be more agile and individually. to deliver our...,194816,[monetary remuneration for employees beyond ba...


In [369]:
#from sklearn.model_selection import train_test_split
#train_query_ids, test_query_ids = train_test_split(df_colbert['query_id'], random_state=42, test_size = 0.2)

In [27]:
with open('../ColBERT/triples.train.tsv', 'wt') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
#    triples_series = df_triple_full[df_triple_full['query_id'].isin(train_query_ids)].query_pos_neg
    triples_series = df_triple_full[df_triple_full['query_id'].isin(train_supplierIds_queryIds)].query_pos_neg
    for i in triples_series.tolist():
        tsv_writer.writerow(i)

In [28]:
len(triples_series.tolist())

151044

In [31]:
# Creating a dataset with all query-passage pairs

import random 

rows_list = []
count = 0
for row in range(len(df_colbert)):
    for i in range(len(df_colbert['passages'].iloc[row])):
        dict_example = {}
        dict_example.update({'query': df_colbert['query'].iloc[row]}) 
        dict_example.update({'query_id': df_colbert['query_id'].iloc[row]}) 
        dict_example.update({'answerPage': df_colbert['answerPage'].iloc[row]}) 
        dict_example.update({'passage': df_colbert['passages'].iloc[row][i][2]})
        dict_example.update({'example_id': count}) 
        count += 1
        rows_list.append(dict_example)

df_binary_full = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'answerPage', 'passage', 'example_id'])      
df_binary_full

,query,query_id,answerPage,passage,example_id
0,detailed information available on fuel consump...,0,[22],—toward the toyota environmental challenge 205...,0
1,detailed information available on fuel consump...,0,[22],editorial policy contents overview of toyota m...,1
2,detailed information available on fuel consump...,0,[22],https://global.toyota/en/sustainability/ envir...,2
3,detailed information available on fuel consump...,0,[22],"report, in which toyota shares with our stakeh...",3
4,detailed information available on fuel consump...,0,[22],subsidiaries and affiliates in japan and overs...,4
...,...,...,...,...,...
2445750,monetary remuneration for employees beyond bas...,8349,[0],2019 stvrs gkn powder metallurgy may 2019 2019...,2445750
2445751,monetary remuneration for employees beyond bas...,8349,[0],again committed to link reward more closely wi...,2445751
2445752,monetary remuneration for employees beyond bas...,8349,[0],2019 mip gkn powder metallurgy may 2019 2019 m...,2445752
2445753,monetary remuneration for employees beyond bas...,8349,[0],be more agile and individually. to deliver our...,2445753


In [32]:
with open('../ColBERT/dev_set.tsv', 'wt') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
#    dev_list = list(zip(df_binary_full[df_binary_full['query_id'].isin(test_query_ids)]['query_id'],\
#                         df_binary_full[df_binary_full['query_id'].isin(test_query_ids)]['example_id'],\
#                         df_binary_full[df_binary_full['query_id'].isin(test_query_ids)]['query'],\
#                         df_binary_full[df_binary_full['query_id'].isin(test_query_ids)]['passage']))
    dev_list = list(zip(df_binary_full[df_binary_full['query_id'].isin(test_supplierIds_queryIds)]['query_id'],\
                         df_binary_full[df_binary_full['query_id'].isin(test_supplierIds_queryIds)]['example_id'],\
                         df_binary_full[df_binary_full['query_id'].isin(test_supplierIds_queryIds)]['query'],\
                         df_binary_full[df_binary_full['query_id'].isin(test_supplierIds_queryIds)]['passage']))
    for i in dev_list:
        tsv_writer.writerow(i)

In [33]:
len(dev_list)

497034

### ColBERT training

In [ ]:
python ../ColBERT/colbert/train.py --amp --doc_maxlen 180 --mask-punctuation --bsize 32 --accum 1 \
--triples ../ColBERT/triples.train.tsv \
--root ../ColBERT/ --experiment ecovadis-1 --similarity l2 --run msmarco.psg.l2

### ColBERT evaluation

In [ ]:
python ../ColBERT/colbert/test.py --amp --doc_maxlen 180 --mask-punctuation \
--topk ../ColBERT/dev_set.tsv
--checkpoint ../ColBERT/ecovadis-1/train.py/msmarco.psg.l2/checkpoints/colbert.dnn \
--root ../ColBERT/ --experiment ecovadis-1

In [18]:
#df_rank_colbert = pd.read_csv("../ColBERT/ecovadis-3/test.py/2021-05-31_21.08.06/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert = pd.read_csv("../ColBERT/ecovadis-10/test.py/2021-06-01_19.49.01/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert

,query_id,passage_id,new_rank
0,8182,2434621,1
1,8182,2434619,2
2,8182,2434624,3
3,8182,2434622,4
4,8182,2434623,5
...,...,...,...
497029,3950,1500869,14
497030,3950,1500877,15
497031,3950,1500871,16
497032,3950,1500870,17


In [19]:
query_goldPage_dict = dict(zip(df_colbert['query_id'], df_colbert['answerPage']))
pass_id_page_dict = {i[1]: i[0] for p in df_colbert['passages'] for i in p}
df_rank_colbert['answerPage'] = df_rank_colbert['query_id'].apply(lambda x: query_goldPage_dict[x])
df_rank_colbert['pagePass'] = df_rank_colbert['passage_id'].apply(lambda x: pass_id_page_dict[x])
df_rank_colbert['pid_rank_passPage'] = df_rank_colbert.apply(lambda x: [x['passage_id'], x['new_rank'], x['pagePass']], axis=1)
df_rank_colbert

,query_id,passage_id,new_rank,answerPage,pagePass,pid_rank_passPage
0,8182,2434621,1,"[0, 1]",0,"[2434621, 1, 0]"
1,8182,2434619,2,"[0, 1]",0,"[2434619, 2, 0]"
2,8182,2434624,3,"[0, 1]",1,"[2434624, 3, 1]"
3,8182,2434622,4,"[0, 1]",0,"[2434622, 4, 0]"
4,8182,2434623,5,"[0, 1]",1,"[2434623, 5, 1]"
...,...,...,...,...,...,...
497029,3950,1500869,14,[2],0,"[1500869, 14, 0]"
497030,3950,1500877,15,[2],2,"[1500877, 15, 2]"
497031,3950,1500871,16,[2],0,"[1500871, 16, 0]"
497032,3950,1500870,17,[2],0,"[1500870, 17, 0]"


In [ ]:
grouped = df_rank_colbert.groupby('query_id')['pid_rank_passPage', 'answerPage']
df_colbert_pred_final = grouped.aggregate(lambda x: list(x))
df_colbert_pred_final

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,pid_rank_passPage,answerPage
query_id,,
75,"[[20526, 1, 20], [20524, 2, 18], [20525, 3, 19...","[[18], [18], [18], [18], [18], [18], [18], [18..."
76,"[[20583, 1, 14], [20600, 2, 31], [20580, 3, 11...","[[14], [14], [14], [14], [14], [14], [14], [14..."
77,"[[20643, 1, 11], [20663, 2, 31], [20646, 3, 14...","[[13], [13], [13], [13], [13], [13], [13], [13..."
78,"[[20705, 1, 10], [20703, 2, 8], [20699, 3, 4],...","[[10], [10], [10], [10], [10], [10], [10], [10..."
79,"[[20777, 1, 19], [20778, 2, 20], [20775, 3, 17...","[[17], [17], [17], [17], [17], [17], [17], [17..."
...,...,...
8318,"[[2443827, 1, 3], [2443813, 2, 1], [2443818, 3...","[[1], [1], [1], [1], [1], [1], [1], [1], [1], ..."
8337,"[[2444502, 1, 19], [2444482, 2, 8], [2444496, ...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
8346,"[[2445737, 1, 3], [2445736, 2, 3], [2445728, 3...","[[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3..."


In [ ]:
# Scoring success on top k results
def predict_pages_colbert(k, page_pass, answer_page):
    prediction = 0
    for j in range(k):
        try:
            if str(page_pass[j][2]) in answer_page[0]:
                prediction = 1 
        except:
            pass
    return prediction

# Running the evaluation on all lines except the ones with 1 page only
for k in [1,3,5,10]:
    results = df_colbert_pred_final.apply(lambda x: predict_pages_colbert(k, x['pid_rank_passPage'], x['answerPage']), axis=1)
    try:
        print("Success @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

In [ ]:
# Investigating the distribution of Success@3 depending on the type of report
import itertools
import numpy as np

colb_query_ids_pred = list(df_colbert_pred_final.index.values)
df_colbert_pred_final2 = df_colbert_pred_final.copy()
df_colbert_pred_final2['typeName'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred)]['typeName']
df_colbert_pred_final2['numPages'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred)]['numPages']
df_colbert_pred_final2 = df_colbert_pred_final2.reset_index()

grouped_type_colb = df_colbert_pred_final2.groupby('typeName')['query_id', 'answerPage', 'numPages', 'pid_rank_passPage']
df_colb_full_type = grouped_type_colb.aggregate(lambda x: list(x))
df_colb_full_type = df_colb_full_type.reset_index()
df_colb_full_type['numPages'] = df_colb_full_type['numPages'].apply(lambda x: np.mean(x))
df_colb_full_type['answerPage'] = df_colb_full_type['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_colb_full_type

colb_types = df_colb_full_type['typeName'].tolist()
print('Success @ 3 for type:')
for s in colb_types:
    index_s_colb = colb_types.index(s)
    query_idx_colb = df_colb_full_type.query_id.iloc[colb_types.index(s)]
    df_types_temp_colb = df_colbert_pred_final2[df_colbert_pred_final2['query_id'].isin(query_idx_colb)]
    results = df_types_temp_colb.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {} with average {:.0f} pages: {:f}".format(s, df_colb_full_type['numPages'].iloc[index_s_colb], sum(results)/len(results)))

In [ ]:
# Scoring MRR on top k results
def eval_MRR_colbert(k, page_pass, answer_page):
    prediction = 0
    stop = False
    for j in range(k):
        try:
            if str(page_pass[j][2]) in answer_page[0] and stop == False:
                prediction = 1/(j+1)
                stop = True
        except:
            pass
    return prediction

# Running the evaluation 
for k in [1,3,5,10]:
    results = df_colbert_pred_final.apply(lambda x: eval_MRR_colbert(k, x['pid_rank_passPage'], x['answerPage']), axis=1)
    try:
        print("MRR @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

In [ ]:
# Investigating the distribution of Success@3 depending on the size of the documents

def page_range(x):
    if x < 31:
        pageRange = '2-30'
    elif x < 61:
        pageRange = '31-60'
    else:
        pageRange = '61-more'
    return pageRange

df_colbert['pageRange'] = df_colbert['numPages'].apply(lambda x: page_range(x))

colb_query_ids_pred = list(df_colbert_pred_final.index.values)
df_colbert_pred_final2 = df_colbert_pred_final.copy()
df_colbert_pred_final2['pageRange'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred)]['pageRange']
df_colbert_pred_final2 = df_colbert_pred_final2.reset_index()

grouped_range_colb = df_colbert_pred_final2.groupby('pageRange')['query_id', 'answerPage', 'pid_rank_passPage']
df_colb_full_range = grouped_range_colb.aggregate(lambda x: list(x))
df_colb_full_range = df_colb_full_range.reset_index()
df_colb_full_range['answerPage'] = df_colb_full_range['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_colb_full_range

colb_ranges = df_colb_full_range['pageRange'].tolist()
print('Success @ 3 for range:')
for s in colb_ranges:
    index_s_colb = colb_ranges.index(s)
    query_idx_colb = df_colb_full_range.query_id.iloc[colb_ranges.index(s)]
    df_ranges_temp_colb = df_colbert_pred_final2[df_colbert_pred_final2['query_id'].isin(query_idx_colb)]
    results = df_ranges_temp_colb.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {}: {:f}".format(s, sum(results)/len(results)))

### Using trained ColBERT to rerank all our passages

In [51]:
import csv
with open('../ColBERT/full_set.tsv', 'wt') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
    dev_list_full = list(zip(df_binary_full['query_id'],\
                         df_binary_full['example_id'],\
                         df_binary_full['query'],\
                         df_binary_full['passage']))
    for i in dev_list_full:
        tsv_writer.writerow(i)

In [52]:
len(dev_list_full)

2445755

In [ ]:
python ../ColBERT/colbert/test.py ---amp --mask-punctuation \
--topk ../ColBERT/full_set.tsv
--checkpoint ../ColBERT/ecovadis-1/train.py/msmarco.psg.l2/checkpoints/colbert.dnn \
--root ../ColBERT/ --experiment ecovadis-1

In [51]:
#df_rank_colbert_full = pd.read_csv("../ColBERT/ecovadis-3/test.py/2021-05-31_21.28.52/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert_full = pd.read_csv("../ColBERT/ecovadis-10/test.py/2021-06-01_20.11.10/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert_full

,query_id,passage_id,new_rank
0,3532,1342549,1
1,3532,1342545,2
2,3532,1342956,3
3,3532,1342828,4
4,3532,1342484,5
...,...,...,...
2445750,6825,2210744,7
2445751,6825,2210745,8
2445752,6825,2210747,9
2445753,6825,2210738,10


In [55]:
query_goldPage_dict = dict(zip(df_colbert['query_id'], df_colbert['answerPage']))
pass_id_page_dict = {i[1]: i[0] for p in df_colbert['passages'] for i in p}
df_rank_colbert_full['answerPage'] = df_rank_colbert_full['query_id'].apply(lambda x: query_goldPage_dict[x])
df_rank_colbert_full['pagePass'] = df_rank_colbert_full['passage_id'].apply(lambda x: pass_id_page_dict[x])
df_rank_colbert_full['pid_rank_passPage'] = df_rank_colbert_full.apply(lambda x: [x['passage_id'], x['new_rank'], x['pagePass']], axis=1)
df_rank_colbert_full

KeyboardInterrupt: 

In [ ]:
grouped_full = df_rank_colbert_full.groupby('query_id')['pid_rank_passPage', 'answerPage']
df_colbert_pred_final_full = grouped_full.aggregate(lambda x: list(x))
df_colbert_pred_final_full

In [ ]:
def pos_passages(top_passages, col_answerPage):
    pos_passages = [top_passages[i] for i in range(len(top_passages)) if top_passages[i][2] in col_answerPage[0]]
    return pos_passages

def neg_passages(top_passages, col_answerPage):
    neg_passages = [top_passages[i] for i in range(len(top_passages)) if top_passages[i][2] not in col_answerPage[0]]
    return neg_passages

df_colbert_pred_final_full['ColPosPassages'] = df_colbert_pred_final_full.apply(lambda x: pos_passages(x['pid_rank_passPage'], x['answerPage']), axis=1)
df_colbert_pred_final_full['ColNegPassages'] = df_colbert_pred_final_full.apply(lambda x: neg_passages(x['pid_rank_passPage'], x['answerPage']), axis=1)
df_colbert_pred_final_full

### Preparing dataset and training via weak supervision

In [118]:
df_colbert['ColPosPassages'] = df_colbert_pred_final_full['ColPosPassages']
df_colbert['ColNegPassages'] = df_colbert_pred_final_full['ColNegPassages']

In [119]:
# Recommended format: TSV file with query text \t positive passage text \t negative passage text
import csv
import random

passId_passText_dict = {i[1]: i[2] for p in df_colbert['passages'] for i in p}

# Creating a dataset with triples: query, positive passage, negative passage

rows_list = []
count = 0
for row in range(len(df_colbert)):
    for i in range(min(3,len(df_colbert['ColPosPassages'].iloc[row]))):
        if len(df_colbert['ColNegPassages'].iloc[row]) > 20:
            neg_samples_idx = random.sample(range(10,len(df_colbert['ColNegPassages'].iloc[row])), min(10, len(df_colbert['ColNegPassages'].iloc[row])))
        else:
            neg_samples_idx = random.sample(range(len(df_colbert['ColNegPassages'].iloc[row])), min(10, len(df_colbert['ColNegPassages'].iloc[row])))
        for j in neg_samples_idx:
            dict_example = {}
            dict_example.update({'query': df_colbert['query'].iloc[row]}) 
            dict_example.update({'query_id': df_colbert['query_id'].iloc[row]}) 
            dict_example.update({'positive': passId_passText_dict[df_colbert['ColPosPassages'].iloc[row][i][0]]})
            dict_example.update({'negative': passId_passText_dict[df_colbert['ColNegPassages'].iloc[row][j][0]]})
            dict_example.update({'example_id': count}) 
            count += 1
            rows_list.append(dict_example)

df_triple_full_col = pd.DataFrame(rows_list, columns = ['query', 'query_id', 'positive', 'negative', 'example_id'])
df_triple_full_col['query_pos_neg'] = df_triple_full_col.apply(lambda x: [x['query'], x['positive'], x['negative']], axis=1)
df_triple_full_col

,query,query_id,positive,negative,example_id,query_pos_neg
0,detailed information available on fuel consump...,0,zero co2 emissions challenge the lca that toyo...,consolidated accounting ・main parts manufactur...,0,[detailed information available on fuel consum...
1,detailed information available on fuel consump...,0,zero co2 emissions challenge the lca that toyo...,one of the medium- to long-term initiatives. w...,1,[detailed information available on fuel consum...
2,detailed information available on fuel consump...,0,zero co2 emissions challenge the lca that toyo...,—toward the toyota environmental challenge 205...,2,[detailed information available on fuel consum...
3,detailed information available on fuel consump...,0,zero co2 emissions challenge the lca that toyo...,efforts to reduce the use of cleaning solvents...,3,[detailed information available on fuel consum...
4,detailed information available on fuel consump...,0,zero co2 emissions challenge the lca that toyo...,below 2°c above pre-industrial levels and purs...,4,[detailed information available on fuel consum...
...,...,...,...,...,...,...
216235,monetary remuneration for employees beyond bas...,8349,again committed to link reward more closely wi...,be more agile and individually. to deliver our...,216235,[monetary remuneration for employees beyond ba...
216236,monetary remuneration for employees beyond bas...,8349,again committed to link reward more closely wi...,2019 mip gkn powder metallurgy may 2019 2019 m...,216236,[monetary remuneration for employees beyond ba...
216237,monetary remuneration for employees beyond bas...,8349,2019 stvrs gkn powder metallurgy may 2019 2019...,be more agile and individually. to deliver our...,216237,[monetary remuneration for employees beyond ba...
216238,monetary remuneration for employees beyond bas...,8349,2019 stvrs gkn powder metallurgy may 2019 2019...,year there will be an equal weighting between ...,216238,[monetary remuneration for employees beyond ba...


In [441]:
#from sklearn.model_selection import train_test_split
#train_query_ids, test_query_ids = train_test_split(df_colbert['query_id'], random_state=42, test_size = 0.2)

In [121]:
with open('../ColBERT/triples_col.train.tsv', 'wt') as f:
    tsv_writer = csv.writer(f, delimiter='\t')
    triples_series = df_triple_full_col[df_triple_full_col['query_id'].isin(train_query_ids)].query_pos_neg
    for i in triples_series.tolist():
        tsv_writer.writerow(i)

In [ ]:
python ../ColBERT/colbert/train.py --amp --doc_maxlen 180 --mask-punctuation --bsize 32 --accum 1 \
--triples ../ColBERT/triples_col.train.tsv \
--root ../ColBERT/ --experiment ecovadis-6 --similarity l2 --run msmarco.psg.l2

### Evaluation of retrained ColBERT

In [ ]:
# Evaluation on the same dev set for comparison with first trained version of ColBERT

python ../ColBERT/colbert/test.py --amp --doc_maxlen 180 --mask-punctuation \
--topk ../ColBERT/dev_set.tsv
--checkpoint ../ColBERT/ecovadis-6/train.py/msmarco.psg.l2/checkpoints/colbert.dnn \
--root ../ColBERT/ --experiment ecovadis-6

In [122]:
#df_rank_colbert_2 = pd.read_csv("../ColBERT/ecovadis-6/test.py/2021-06-01_11.26.45/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert_2 = pd.read_csv("../ColBERT/ecovadis-11/test.py/2021-06-01_22.17.08/ranking.tsv", sep='\t', names=['query_id', 'passage_id', 'new_rank'])
df_rank_colbert_2

,query_id,passage_id,new_rank
0,8182,2434621,1
1,8182,2434619,2
2,8182,2434624,3
3,8182,2434623,4
4,8182,2434622,5
...,...,...,...
497029,3950,1500871,14
497030,3950,1500883,15
497031,3950,1500881,16
497032,3950,1500869,17


In [123]:
query_goldPage_dict = dict(zip(df_colbert['query_id'], df_colbert['answerPage']))
pass_id_page_dict = {i[1]: i[0] for p in df_colbert['passages'] for i in p}
df_rank_colbert_2['answerPage'] = df_rank_colbert_2['query_id'].apply(lambda x: query_goldPage_dict[x])
df_rank_colbert_2['pagePass'] = df_rank_colbert_2['passage_id'].apply(lambda x: pass_id_page_dict[x])
df_rank_colbert_2['pid_rank_passPage'] = df_rank_colbert_2.apply(lambda x: [x['passage_id'], x['new_rank'], x['pagePass']], axis=1)
df_rank_colbert_2

,query_id,passage_id,new_rank,answerPage,pagePass,pid_rank_passPage
0,8182,2434621,1,"[0, 1]",0,"[2434621, 1, 0]"
1,8182,2434619,2,"[0, 1]",0,"[2434619, 2, 0]"
2,8182,2434624,3,"[0, 1]",1,"[2434624, 3, 1]"
3,8182,2434623,4,"[0, 1]",1,"[2434623, 4, 1]"
4,8182,2434622,5,"[0, 1]",0,"[2434622, 5, 0]"
...,...,...,...,...,...,...
497029,3950,1500871,14,[2],0,"[1500871, 14, 0]"
497030,3950,1500883,15,[2],3,"[1500883, 15, 3]"
497031,3950,1500881,16,[2],2,"[1500881, 16, 2]"
497032,3950,1500869,17,[2],0,"[1500869, 17, 0]"


In [124]:
import itertools
grouped_2 = df_rank_colbert_2.groupby('query_id')['query_id', 'pid_rank_passPage', 'answerPage']
df_colbert_pred_final_2 = grouped_2.aggregate(lambda x: list(x))
df_colbert_pred_final_2['query_id'] = df_colbert_pred_final_2['query_id'].apply(lambda x: int(str(list(set(x))).strip('[]')))
df_colbert_pred_final_2['answerPage'] = df_colbert_pred_final_2['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_colbert_pred_final_2

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,query_id,pid_rank_passPage,answerPage
query_id,,,
75,75,"[[20524, 1, 18], [20526, 2, 20], [20525, 3, 19...",[[18]]
76,76,"[[20583, 1, 14], [20581, 2, 12], [20580, 3, 11...",[[14]]
77,77,"[[20643, 1, 11], [20646, 2, 14], [20644, 3, 12...",[[13]]
78,78,"[[20705, 1, 10], [20703, 2, 8], [20699, 3, 4],...",[[10]]
79,79,"[[20777, 1, 19], [20778, 2, 20], [20775, 3, 17...",[[17]]
...,...,...,...
8318,8318,"[[2443814, 1, 1], [2443813, 2, 1], [2443827, 3...",[[1]]
8337,8337,"[[2444492, 1, 14], [2444494, 2, 14], [2444496,...","[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
8346,8346,"[[2445728, 1, 1], [2445736, 2, 3], [2445725, 3...","[[2, 3]]"


In [125]:
# Scoring success on top k results
def predict_pages_colbert(k, page_pass, answer_page):
    prediction = 0
    for j in range(k):
        try:
            if page_pass[j][2] in answer_page[0]:
                prediction = 1 
        except:
            pass
    return prediction

# Running the evaluation on all lines except the ones with 1 page only
for k in [1,3,5,10]:
    results = df_colbert_pred_final_2.apply(lambda x: predict_pages_colbert(k, x['pid_rank_passPage'], x['answerPage']), axis=1)
    try:
        print("Success @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

Success @ 1: 0.558370
Success @ 3: 0.747797
Success @ 5: 0.813877
Success @ 10: 0.874449


In [126]:
# Investigating the distribution of Success@3 depending on the type of report
import itertools

colb_query_ids_pred2 = list(df_colbert_pred_final_2.index.values)
df_colbert_pred_final3 = df_colbert_pred_final2.copy()
df_colbert_pred_final3['typeName'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred2)]['typeName']
df_colbert_pred_final3['numPages'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred2)]['numPages']
df_colbert_pred_final3 = df_colbert_pred_final3.reset_index()

grouped_type_colb2 = df_colbert_pred_final3.groupby('typeName')['query_id', 'answerPage', 'numPages', 'pid_rank_passPage']
df_colb_full_type2 = grouped_type_colb2.aggregate(lambda x: list(x))
df_colb_full_type2 = df_colb_full_type2.reset_index()
df_colb_full_type2['numPages'] = df_colb_full_type2['numPages'].apply(lambda x: np.mean(x))
df_colb_full_type2['answerPage'] = df_colb_full_type2['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])
df_colb_full_type2

colb_types2 = df_colb_full_type2['typeName'].tolist()
print('Success @ 3 for type:')
for s in colb_types2:
    index_s_colb2 = colb_types2.index(s)
    query_idx_colb2 = df_colb_full_type2.query_id.iloc[colb_types2.index(s)]
    df_types_temp_colb2 = df_colbert_pred_final3[df_colbert_pred_final3['query_id'].isin(query_idx_colb2)]
    results = df_types_temp_colb2.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {} with average {:.0f} pages: {:f}".format(s, df_colb_full_type2['numPages'].iloc[index_s_colb2], sum(results)/len(results)))

Success @ 3 for type:
  Annual_Report with average 118 pages: 0.691275
  CSR_Report with average 63 pages: 0.723684
  Employee_Handbook with average 81 pages: 1.000000
  Procedure with average 4 pages: 1.000000


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  # Remove the CWD from sys.path while we load stuff.


In [127]:
# Scoring MRR on top k results
def eval_MRR_colbert(k, page_pass, answer_page):
    prediction = 0
    stop = False
    for j in range(k):
        try:
            if page_pass[j][2] in answer_page[0] and stop == False:
                prediction = 1/(j+1)
                stop = True
        except:
            pass
    return prediction

# Running the evaluation 
for k in [1,3,5,10]:
    results = df_colbert_pred_final_2.apply(lambda x: eval_MRR_colbert(k, x['pid_rank_passPage'], x['answerPage']), axis=1)
    try:
        print("MRR @ {}: {:f}".format(k, sum(results)/len(results)))
    except:
        print('Error, try lower k')

MRR @ 1: 0.558370
MRR @ 3: 0.642621
MRR @ 5: 0.657764
MRR @ 10: 0.666097


In [129]:
# Investigating the distribution of Success@3 depending on the size of the documents

def page_range(x):
    if x < 31:
        pageRange = '2-30'
    elif x < 61:
        pageRange = '31-60'
    else:
        pageRange = '61-more'
    return pageRange

df_colbert['pageRange'] = df_colbert['numPages'].apply(lambda x: page_range(x))

colb_query_ids_pred2b = list(df_colbert_pred_final_2.index.values)
df_colbert_pred_final3b = df_colbert_pred_final2.copy()
df_colbert_pred_final3b['typeName'] = df_colbert[df_colbert['query_id'].isin(colb_query_ids_pred2b)]['pageRange']
df_colbert_pred_final3b = df_colbert_pred_final3b.reset_index()

grouped_type_colb2b = df_colbert_pred_final3b.groupby('typeName')['query_id', 'answerPage', 'pid_rank_passPage']
df_colb_full_type2b = grouped_type_colb2b.aggregate(lambda x: list(x))
df_colb_full_type2b = df_colb_full_type2b.reset_index()
df_colb_full_type2b['answerPage'] = df_colb_full_type2b['answerPage'].apply(lambda x: [i for i,_ in itertools.groupby(x)])

colb_ranges2 = df_colb_full_type2b['typeName'].tolist()
print('Success @ 3 for range:')
for s in colb_ranges2:
    index_s_colb2 = colb_ranges2.index(s)
    query_idx_colb2 = df_colb_full_type2.query_id.iloc[colb_ranges2.index(s)]
    df_types_temp_colb2 = df_colbert_pred_final3b[df_colbert_pred_final3b['query_id'].isin(query_idx_colb2)]
    results = df_types_temp_colb2.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("  {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for range:
  2-30: 0.691275
  31-60: 0.723684
  61-more: 1.000000


/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:19: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


### Ablation studies

In [83]:
import numpy as np
grouped_size = df_colbert.groupby('size')['query_id', 'answerPage', 'numPages']
df_colbert_size = grouped_size.aggregate(lambda x: list(x))
df_colbert_size = df_colbert_size.reset_index()
df_colbert_size['numPages'] = df_colbert_size['numPages'].apply(lambda x: np.mean(x))
df_colbert_size

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


,size,query_id,answerPage,numPages
0,L,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[22], [50], [31], [20], [12], [51], [26, 54],...",75.488506
1,M,"[75, 76, 77, 78, 79, 201, 202, 221, 222, 223, ...","[[18], [14], [13], [10], [17], [6], [6], [12],...",51.017328
2,S,"[107, 108, 109, 110, 111, 112, 113, 114, 115, ...","[[8], [8], [7], [14], [19], [19], [17], [17, 5...",40.926923
3,XS,"[164, 165, 166, 167, 168, 2333, 2334, 2335, 23...","[[6, 7], [0], [1, 5], [4, 6, 7], [2, 3], [14],...",30.053498


In [126]:
sizes = df_colbert_size['size'].tolist()
for s in sizes:
    query_idx = df_colbert_size.query_id.iloc[sizes.index(s)]
    df_sizes_temp = df_colbert_pred_final_2[df_colbert_pred_final_2['query_id'].isin(query_idx)]
    results = df_sizes_temp.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    print("Success @ 3 for size {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for size L: 0.700368
Success @ 3 for size M: 0.772622
Success @ 3 for size S: 0.762376
Success @ 3 for size XS: 0.680000


In [85]:
grouped_type = df_colbert.groupby('typeName')['query_id', 'answerPage', 'numPages']
df_colbert_type = grouped_type.aggregate(lambda x: list(x))
df_colbert_type = df_colbert_type.reset_index()
df_colbert_type['numPages'] = df_colbert_type['numPages'].apply(lambda x: np.mean(x))
df_colbert_type

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,typeName,query_id,answerPage,numPages
0,Annual_Report,"[80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 9...","[[34, 27], [27], [90], [53], [53], [41, 95], [...",97.171956
1,Audit_Report,"[5207, 5208, 5209, 5210, 5211, 5212, 5213, 521...","[[24], [20], [23], [20], [28], [32], [26], [26...",28.487805
2,CSR_Report,"[21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 3...","[[38], [34], [34], [27], [27], [27], [21], [41...",71.816555
3,Certificate,"[2377, 3875, 4176, 4190, 4216, 4217, 4244, 530...","[[0], [0, 1, 2, 3, 4, 5, 6, 7, 8], [0, 1, 2, 3...",8.625000
4,Code_of_Ethics_Code_of_Conduct,"[898, 899, 900, 2350, 2351, 2352, 2553, 2554, ...","[[], [], [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...",19.844498
5,Employee_Handbook,"[431, 432, 433, 434, 435, 436, 437, 438, 453, ...","[[64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, ...",56.364130
6,Letter_to_EcoVadis,"[5236, 5237, 5238, 5239, 5240, 5241, 5242]","[[66], [21], [44], [25], [29, 30], [12], [39]]",82.000000
7,Other,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[22], [50], [31], [20], [12], [51], [26, 54],...",43.613583
8,Policy,"[439, 440, 441, 442, 443, 444, 445, 446, 447, ...","[[42], [42], [14], [21], [20], [23], [32], [15...",29.066964
9,Procedure,"[169, 171, 630, 631, 1665, 1666, 1667, 1780, 1...","[[], [], [], [], [0, 1, 2], [10], [12], [0, 1,...",16.050388


In [130]:
types = df_colbert_type['typeName'].tolist()
for s in types:
    query_idx = df_colbert_type.query_id.iloc[types.index(s)]
    df_types_temp = df_colbert_pred_final_2[df_colbert_pred_final_2['query_id'].isin(query_idx)]
    results = df_types_temp.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    if len(results) != 0:
        print("Success @ 3 for type {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for type Annual_Report: 0.720930
Success @ 3 for type Audit_Report: 0.750000
Success @ 3 for type CSR_Report: 0.688604
Success @ 3 for type Certificate: 0.666667
Success @ 3 for type Code_of_Ethics_Code_of_Conduct: 0.817073
Success @ 3 for type Employee_Handbook: 0.748768
Success @ 3 for type Other: 0.659091
Success @ 3 for type Policy: 0.742647
Success @ 3 for type Procedure: 0.854545
Success @ 3 for type Supplier_Code_of_Conduct: 0.666667
Success @ 3 for type Webpage_Intranet_extract: 0.833333


In [75]:
grouped_theme = df_colbert.groupby('theme')['query_id', 'answerPage', 'numPages']
df_colbert_theme = grouped_theme.aggregate(lambda x: list(x))
df_colbert_theme = df_colbert_theme.reset_index()
df_colbert_theme['numPages'] = df_colbert_theme['numPages'].apply(lambda x: np.mean(x))
df_colbert_theme

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,theme,query_id,answerPage,numPages
0,ENV,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[[22], [50], [31], [20], [12], [51], [26, 54],...",70.006418
1,FBP,"[113, 114, 115, 120, 145, 155, 156, 157, 159, ...","[[17], [17, 5], [17], [5, 14], [73], [27], [27...",52.208782
2,Gen,"[47, 68, 73, 74, 96, 97, 104, 105, 106, 121, 1...","[[1], [61], [2], [24, 23], [6], [9], [86], [88...",65.830325
3,LAB,"[24, 25, 26, 31, 32, 33, 34, 35, 45, 46, 48, 5...","[[27], [27], [27], [42], [42], [41, 42], [42],...",67.016263
4,SUP,"[27, 222, 223, 224, 239, 318, 411, 485, 512, 5...","[[21], [19], [11], [11], [62], [7], [47], [10]...",64.639785


In [125]:
themes = df_colbert_theme['theme'].tolist()
for s in themes:
    query_idx = df_colbert_theme.query_id.iloc[themes.index(s)]
    df_types_theme = df_colbert_pred_final_2[df_colbert_pred_final_2['query_id'].isin(query_idx)]
    results = df_types_theme.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    print("Success @ 3 for type {}: {:f}".format(s, sum(results)/len(results)))

Success @ 3 for type ENV: 0.725417
Success @ 3 for type FBP: 0.737778
Success @ 3 for type Gen: 0.653543
Success @ 3 for type LAB: 0.730370
Success @ 3 for type SUP: 0.701923


In [132]:
grouped_numPages = df_colbert.groupby('numPages')['query_id', 'answerPage']
df_colbert_numPages = grouped_numPages.aggregate(lambda x: list(x))
df_colbert_numPages = df_colbert_numPages.reset_index()
df_colbert_numPages

/anaconda/envs/pygaggle2/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,numPages,query_id,answerPage
0,2.0,"[627, 628, 629, 693, 897, 898, 899, 1417, 2348...","[[0], [0], [0], [0], [0], [], [], [0], [0], [0..."
1,3.0,"[1665, 1757, 1835, 1858, 1860, 1861, 1862, 186...","[[0, 1, 2], [], [], [0, 1, 2], [1], [1], [0], ..."
2,4.0,"[169, 630, 631, 1780, 1857, 2086, 2165, 2166, ...","[[], [], [], [0, 1, 2, 3], [0, 1, 2], [0, 1, 2..."
3,5.0,"[1873, 2087, 2377, 2555, 3585, 3892, 4437, 446...","[[0, 1, 2, 3, 4], [0, 1, 2, 3, 4], [0], [0, 1,..."
4,6.0,"[1874, 4458, 4805, 5151, 5184, 5185, 5221, 522...","[[0, 1, 2, 3, 4, 5], [], [1], [0, 1, 2], [1], ..."
...,...,...,...
163,264.0,"[5862, 5863, 5864, 5865, 5866, 5867, 5868, 586...","[[101], [108], [101], [99], [97, 98], [98], [9..."
164,266.0,[8056],"[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13..."
165,308.0,"[5477, 5478, 5479, 5480]","[[41], [233], [35, 15, 16, 17, 18], [16, 15]]"
166,333.0,"[5550, 5551, 5552, 5553, 5554, 5555, 5556, 555...","[[158], [246], [251, 220], [172], [55], [297],..."


In [133]:
numPages = df_colbert_numPages['numPages'].tolist()
for s in numPages:
    query_idx = df_colbert_numPages.query_id.iloc[sizes.index(s)]
    df_sizes_temp = df_colbert_pred_final_2[df_colbert_pred_final_2['query_id'].isin(query_idx)]
    results = df_sizes_temp.apply(lambda x: predict_pages_colbert(3, x['pid_rank_passPage'], x['answerPage']), axis=1)
    print("Success @ 3 for size {}: {:f}".format(s, sum(results)/len(results)))

ValueError: 2.0 is not in list